In [ ]:
import re
import numpy as np
from PIL import Image

from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.layers import Activation
from keras.layers import Input, Lambda, Dense, Dropout, Convolution2D, MaxPooling2D, Flatten
from keras.models import Sequential, Model
from tensorflow.keras.optimizers import RMSprop,Adam
from keras import optimizers

import matplotlib.image as mpimg 
import matplotlib.pyplot as plt 

from keras import callbacks
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, EarlyStopping, ReduceLROnPlateau, TensorBoard
import os
from keras.models import Model,load_model
import json
from keras.models import model_from_json, load_model
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D, BatchNormalization
import warnings
warnings.filterwarnings('ignore')

In [ ]:
#Path of testing directory
path =  os.path.join('xxxx')

In [ ]:
#loading the model
from keras.models import model_from_json
json_file = open('/content/drive/MyDrive/Major Project/Training Files/Dataset1/One Shot Learning/Model/model_architecture (1).json', 'r')
loaded_model_json = json_file.read()
json_file.close()
loaded_model = model_from_json(loaded_model_json)
# load weights into new model
loaded_model.load_weights("/content/drive/MyDrive/Major Project/Training Files/Dataset1/One Shot Learning/Model/model_weights.h5")
print("Loaded model from disk")

In [ ]:
loaded_model.summary()

In [ ]:
#defining a function to generate pairs for n way one shot learning
def n_way_pairs(n):
    #read the image
    img = mpimg.imread(path+'S' + str(1) + '/' + str(1) + '_100.jpg', 'rw+')
    #get the new size
    dim1 = image.shape[0]
    dim2 = image.shape[1]

    count = 0

    #initialize the numpy array with the shape of [total_sample, no_of_pairs, dim1, dim2]
    x_geuine_pair = np.zeros([1, 2, 1, dim1, dim2])

    y_genuine = np.zeros([1,1])

    ind1=np.random.randint(1,folder_count)
    image_index1=np.random.randint(1,image_count)
    image_index2=np.random.randint(1,image_count)

    image1= mpimg.imread(path+'S'+str(ind1)+'/'+str(image_index1)+'_100.jpg','rw+')
    image2 = mpimg.imread(path+'S'+str(ind1)+'/'+str(image_index2)+'_100.jpg','rw+')

    x_geuine_pair[0, 0, 0, :, :] = img1
    x_geuine_pair[0, 1, 0, :, :] = img2
    y_genuine[0] = 1

    x_imposite_pair = np.zeros([n-1, 2, 1, dim1, dim2])
    y_imposite = np.zeros([n-1, 1])

    count=0

    for i in range(n-1):
      while True:
        ind2=np.random.randint(1,folder_count)
        if ind1 != ind2:
          break
      image_index3=np.random.randint(1,image_count)

      image3=mpimg.imread(path+'S'+str(ind2)+'/'+str(image_index3)+'_100.jpg','rw+')
      
      x_imposite_pair[count, 0, 0, :, :] = img1
      x_imposite_pair[count, 1, 0, :, :] = img3
      #as we are drawing images from the different directory we assign label as 0. (imposite pair)
      y_imposite[count] = 0
      count += 1

    #now, concatenate, genuine pairs and imposite pair to get the whole data
    #print(x_geuine_pair.shape)
    #print(x_imposite_pair.shape)
    X = np.concatenate([x_geuine_pair, x_imposite_pair], axis=0)/255
    Y = np.concatenate([y_genuine, y_imposite], axis=0)

    return X, Y

In [ ]:
#defining a function to calculate the accuracy in n way one shot learning
def n_way_accuracy(steps,n):
  k=0
  for i in range(steps):
    x_test,y_test=n_way_pairs(n)
    preds=[]
    for j in range(n):
      pred=loaded_model.predict([x_test[j:j+1, 0], x_test[j:j+1, 1]])
      preds.append(pred)
    preds=np.array(preds)
    actual_index=np.argmax(np.array(y_test))
    predicted_index=np.argmin(preds)
    if actual_index==predicted_index:
      k+=1
  accuracy=(k/steps)*100
  
  return accuracy

In [ ]:
steps=100 #number of steps for n way one shot learning
history=[] #initialize the list to store accuracies 
for i in range(5,40,5):
  n_accuracy=n_way_accuracy(steps,i)
  history.append(n_accuracy)
  print(f"The accuracy for {i} way learning is :{n_accuracy}")

In [ ]:
#plotting the graph
ns=[5,10,15,20,25,30,35]
plt.plot(ns, history)
plt.xlabel("N in N-Way One shot learning")
plt.ylabel("Accuracy")
plt.title('Variation of accuracy by varying n in N-Way one shot learning')

plt.show()